In [6]:
import ase
from ase.build import bulk
from ase.lattice.cubic import FaceCenteredCubic
from ase.calculators import lj, morse
import chemiscope as chemi
import os
import numpy as np

## Make a XYZ file ##

In [7]:
Atom = 'Cu'
Structure = 'fcc'

# Lattice parameters:
r_lat0 = 3.60 # [Angstrom]
E_co = 3.38e5 # [Joules]
Bulk_modulus = 133 # [GPa]
# Results from DFT calculations:
epsilon = 6.567 # [Joules]
sigma = 2.334e-10 # [Meters]

# Generate a directory for the atom:
if not os.path.exists(Atom):
    os.makedirs(Atom)

# Generate the ASE data:
aseBulkData = bulk(
    Atom, 
    Structure, 
    a=r_lat0,
    cubic=True
)
aseLatticeData = FaceCenteredCubic(
    directions=[[1,-1,0], [1,1,-2], [1,1,1]],
    size=(2,2,3),
    symbol='Cu',
    pbc=(1,1,0)
)

# Write the ASE data to a XYZ file:
aseBulkData.write(f'{Atom}/{Atom}_bulk_{Structure}.xyz')
aseLatticeData.write(f'{Atom}/{Atom}_lattice_{Structure}.xyz')

## Generate defects (to do) ##

In [8]:
# Duplicate the XYZ file
# os.system(f'cp {Atom}/{Atom}_{Structure}.xyz {Atom}/{Atom}_{Structure}_defect.xyz')

# Generate random defects in the XYZ file:
# TO DO #

## Calculate Potentials ##

In [9]:
frames = ase.io.read(f'{Atom}/{Atom}_lattice_{Structure}.xyz', ':')

Charges = aseLatticeData.get_atomic_numbers()

aseLatticeData.calc = lj.LennardJones(sigma=sigma, epsilon=epsilon)
LennardJonesData = aseLatticeData.get_potential_energies()
LennardJonesDataGlobal = aseLatticeData.get_potential_energy()

aseLatticeData.calc = morse.MorsePotential()
# Not implemented: MorseData = aseBulkData.get_potential_energies()
MorseDataGlobal = aseLatticeData.get_potential_energy()

properties = {
    "LennardJones": {
        "target": "atom",
        "values": LennardJonesData,
        "units": "J",
    },
    "LennardJonesGlobal": {
        "target": "structure",
        "values": np.array([LennardJonesDataGlobal]),
        "units": "J",
    },
    "AtomicNumbers": {
        "target": "atom",
        "values": Charges,
        "units": "J",
    },
    "MorseGlobal": {
        "target": "structure",
        "values": np.array([MorseDataGlobal]),
        "units": "J",
    },
}

print(properties)

{'LennardJones': {'target': 'atom', 'values': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0.]), 'units': 'J'}, 'LennardJonesGlobal': {'target': 'structure', 'values': array([0.]), 'units': 'J'}, 'AtomicNumbers': {'target': 'atom', 'values': array([29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
       29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
       29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
       29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
       29, 29, 29, 29]), 'units': 'J'}, 'MorseGlobal': {'target': 'structure', 'values': array([-0.00341323]), 'units': 'J'}}


In [10]:
DatasetInformation = {
    'name': Atom + ' ' + Structure,
    'description': 'Semester Project 1, atom ' + Atom + ' in ' + Structure + ' structure',
    'authors': [
        'Maxime Saillen',
    ],
    'references': [
        'https://doi.org/10.1016/j.commatsci.2022.111206',
    ],
}

chemi.write_input(
    path=f"{Atom}/{Atom}.json.gz",
    frames=frames, 
    meta=DatasetInformation, 
    properties=properties, 
    environments=chemi.all_atomic_environments(frames), 
    settings=None, 
    shapes=None, 
    parameters=None
)